In [1]:
import torch
import pandas as pd
import plotly.express as px

from tqdm.auto import tqdm
from nnsight import LanguageModel
from src.utils import path_patch, plain_run

In [2]:
gemma = LanguageModel("google/gemma-2-2b", torch_dtype=torch.bfloat16, device_map="cuda")
gemma.model.train()

Gemma2Model(
  (embed_tokens): Embedding(256000, 2304, padding_idx=0)
  (layers): ModuleList(
    (0-25): 26 x Gemma2DecoderLayer(
      (self_attn): Gemma2Attention(
        (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
        (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
        (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        (rotary_emb): Gemma2RotaryEmbedding()
      )
      (mlp): Gemma2MLP(
        (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
        (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
        (act_fn): PytorchGELUTanh()
      )
      (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
      (pre_feedforward_layernorm): Gemma2RMSNorm((2304,), e

In [3]:
[k for k, v in gemma.model.named_modules()]

['.model',
 '.model.embed_tokens',
 '.model.layers',
 '.model.layers.0',
 '.model.layers.0.self_attn',
 '.model.layers.0.self_attn.q_proj',
 '.model.layers.0.self_attn.k_proj',
 '.model.layers.0.self_attn.v_proj',
 '.model.layers.0.self_attn.o_proj',
 '.model.layers.0.self_attn.rotary_emb',
 '.model.layers.0.mlp',
 '.model.layers.0.mlp.gate_proj',
 '.model.layers.0.mlp.up_proj',
 '.model.layers.0.mlp.down_proj',
 '.model.layers.0.mlp.act_fn',
 '.model.layers.0.input_layernorm',
 '.model.layers.0.post_attention_layernorm',
 '.model.layers.0.pre_feedforward_layernorm',
 '.model.layers.0.post_feedforward_layernorm',
 '.model.layers.1',
 '.model.layers.1.self_attn',
 '.model.layers.1.self_attn.q_proj',
 '.model.layers.1.self_attn.k_proj',
 '.model.layers.1.self_attn.v_proj',
 '.model.layers.1.self_attn.o_proj',
 '.model.layers.1.self_attn.rotary_emb',
 '.model.layers.1.mlp',
 '.model.layers.1.mlp.gate_proj',
 '.model.layers.1.mlp.up_proj',
 '.model.layers.1.mlp.down_proj',
 '.model.layers.

In [12]:
patched_logits = path_patch(
  gemma, 
  [(".model.layers.25.post_attention_layernorm", -1),
   (".model.layers.25.post_feedforward_layernorm", -1)], 
  [".model.norm"],
  "goodbye",
  "goodbye"
)

clean_logits = plain_run(gemma, "goodbye")

In [13]:
(patched_logits - clean_logits).max()

tensor(0., device='cuda:0', dtype=torch.bfloat16, grad_fn=<MaxBackward1>)